# Analysing Different Methods of Finding $Q^2$ and $x$

## Finding $x$ and $Q^2$ using eletron scattering and the final hadronic state

In [74]:
import numpy as np
import uproot as ur
import awkward as ak
import matplotlib.pyplot as plt
# 18X275 or 5x41
E, Ep = 5, 41
s = 4 * E * Ep
xe, xjb, xdiff, epz = [], [], [], []


eAngle, eEnergy, epz, pth = [], [], [], []

Q2e, Q2jb = [],[]
fileCount = 5

# indexes of x we used
indexes = [[] for i in range(fileCount)]

# get data from file

files = list(f'../../data/pythia8NCDIS_{E}x{Ep}_minQ2=10_beamEffects_xAngle=-0.025_hiDiv_1.084{i}.eicrecon.tree.edm4eic.root' for i in range(fileCount))

for f, file_path in enumerate(files):
    events = ur.open(file_path + ':events')
    reconstructed_charged_particles = events['ReconstructedChargedParticles'].arrays()

    # loop through events to find electrons and store their momentum
    for i, event in  enumerate(reconstructed_charged_particles['ReconstructedChargedParticles.PDG']):
        eh, pzh, pyh, pxh, hadrons = 0, 0, 0, 0, 0
        
        xevent = []
        Q2event = []

        angleEvents = []
        energyEvents = []
        
        # if its just an electron, remove it
        if 1 == len(event):
            continue

        
        for j, particle in  enumerate(event):
            # if its an electron, find with scattered electron method
            if particle == 11:
                
                
                m = reconstructed_charged_particles['ReconstructedChargedParticles.mass'][i][j]
                kp1, kp2, kp3 = reconstructed_charged_particles['ReconstructedChargedParticles.momentum.x'][i][j], reconstructed_charged_particles['ReconstructedChargedParticles.momentum.y'][i][j], reconstructed_charged_particles['ReconstructedChargedParticles.momentum.z'][i][j]
                
                # check psuedo-rapidity
                kp = np.sqrt(kp1**2 + kp2**2 + kp3**2)
                theta = np.arccos(kp3/kp)
                pr = -np.log(np.tan(theta / 2))
                
                # remove particles with -4 < psuedo-rapidity < 4
                if abs(pr) < 4:

                    theta = np.arctan2(np.sqrt(kp1**2 + kp2**2), kp3)

                    k3 = -E
                    m0 = 0.000511
                    kp0 = np.sqrt(m0**2+(kp1**2+kp2**2+kp3**2))
                    k0 = np.sqrt(m0**2 + k3**2)
                    q0 = k0 - kp0
                    q1 =    - kp1
                    q2 =    - kp2
                    q3 = k3 - kp3
                    Q2 =-(q0**2 - q1**2 - q2**2 - q3**2) 
                    Q2event.append(Q2)


                    alpha = -0.025
                    p1 = Ep * np.sin(alpha)
                    p2 = 0
                    p3 = Ep * np.cos(alpha)
                    p0 = np.sqrt(0.938**2 + p1**2 + p2**2 + p3**2)
                    pq = p0 * q0 - p1 * q1 - p2 * q2 - p3 * q3
                    xevent.append(0.5 * Q2 / pq)
                    energyEvents.append(Eproton)
                    angleEvents.append(theta)
                    
                    
                    
            # get the sum of components from hadrons for JB method
            elif abs(particle) == 211 or abs(particle) == 321 or particle == 2212:
                
                
                m = reconstructed_charged_particles['ReconstructedChargedParticles.mass'][i][j]
                kp1, kp2, kp3 = reconstructed_charged_particles['ReconstructedChargedParticles.momentum.x'][i][j], reconstructed_charged_particles['ReconstructedChargedParticles.momentum.y'][i][j], reconstructed_charged_particles['ReconstructedChargedParticles.momentum.z'][i][j]
                
                
                kp = np.sqrt(kp1**2 + kp2**2 + kp3**2)
                theta = np.arccos(kp3/kp)
                pr = -np.log(np.tan(theta / 2))
                
                if abs(pr) < 4:
                    hadrons += 1
                    eh  += np.sqrt(m**2+(kp1**2+kp2**2+kp3**2))
                    pxh += kp1
                    pyh += kp2
                    pzh += kp3
                
        
        # add one per event
        y = 1/(2 * E) * (eh - pzh)
        Q2 = 1/(1 - y) * (pxh**2 + pyh**2)
        
        if hadrons != 0 and len(Q2event) != 0:
            indexes[f].append(i)
            epz.append((eh - pzh))
            pth.append((pxh**2 + pyh**2))
            eAngle.append(angleEvents[Q2event.index(max(Q2event))])
            eEnergy.append(energyEvents[Q2event.index(max(Q2event))])


            xjb.append(Q2 / (s * y))
            xe.append(xevent[Q2event.index(max(Q2event))])

            Q2jb.append(Q2)
            Q2e.append(max(Q2event))



## Finding the true $x$ and $Q^2$

In [75]:
mc_particles = events['MCParticles'].arrays()

Q2true, xtrue = [], []
# loop through all the indexes we got the other values from so we skip the same events
for lst  in indexes:
    for i in lst:
        event = mc_particles['MCParticles.PDG'][i]
        Q2tot, xtot = [], []
        for j, particle in enumerate(event):
            if mc_particles['MCParticles.generatorStatus'][i][j] != 1 or particle != 11:
                continue
            
            psx, psy, psz = mc_particles['MCParticles.momentum.x'][i][j], mc_particles['MCParticles.momentum.y'][i][j], mc_particles['MCParticles.momentum.z'][i][j]
            Eproton = np.sqrt(0.000511**2 + psx**2 + psy**2 + psz**2)
            theta = np.arctan2(np.sqrt(psx**2 + psy**2), psz)
            
            Q2 = 2 * E * Eproton * (1 + np.cos(theta))
            
            y = 1 - 0.5 * Eproton / E * (1 - np.cos(theta))
            x = Q2 / s / y
            xtot.append(x)
            Q2tot.append(Q2)

        # if we have multiple events, use one with largest Q2
        if len(xtot) > 0:
            xtrue.append(xtot[Q2tot.index(max(Q2tot))])
            Q2true.append(max(Q2tot))

print(len(xtrue))

5085


In [76]:
import math

for i in range(len(xtrue)):
    q2 = 2 * eEnergy[i] * E * (1 + math.cos(eAngle[i]))

    print(q2, Q2true[i], Q2e[i])

19.65096229718406 22.086264108179666 19.81410523212918
20.981041346638634 41.84260760267922 24.633362520769552
11.052356317149979 31.470511819657208 11.948260006244968
24.63678885960188 46.25981044604201 12.848273295658634
38.489225122585474 10.34404981107506 30.944877818092394
26.33647979648507 17.78808418030362 28.86252690729621
58.32344669967777 28.06853233676772 17.812456735446126
57.95823488995752 14.43793571973954 41.76960526210116
14.537607234294084 26.620863640266275 13.937906618597047
13.790507351411348 31.72236352680893 14.76886414770733
9.960440610250695 42.560353559936736 10.757807760336405
12.783265168990141 11.83186037507965 14.332895999686873
26.055687642161537 84.53093178470836 29.374555394608038
30.0453030575787 17.780777022443427 29.78720769619638
9.538880584258315 10.642652091617192 9.483738993945448
44.34614371083068 11.105397991652993 28.286486903213785
10.038402582706716 15.130014990982765 10.593429486488333
19.752511878390358 11.764163591295128 12.863955595691746

## Training Artificial Neural Network

In [77]:
import torch
import torch.nn.functional as F 
import torch.nn as nn



# Define neural network 
class Network(nn.Module): 
   def __init__(self, input_size, output_size): 
       super(Network, self).__init__() 
        
       self.layer1 = nn.Linear(input_size, 24) 
       self.layer2 = nn.Linear(24, 24) 
       self.layer3 = nn.Linear(24, 12) 
       self.layer4 = nn.Linear(12, output_size) 


   def forward(self, x): 
       x1 = F.relu(self.layer1(x)) 
       x2 = F.relu(self.layer2(x1)) 
       x3 = F.relu(self.layer3(x2)) 
       x4 = self.layer4(x3) 
       return x4

  # Train the model
def train(model, indexes, loss_fn, optimizer, num_epochs):
    for epoch in range(num_epochs):
      print(epoch)
      for i in indexes:
        input_sequence = torch.Tensor([eAngle[i], pth[i], eEnergy[i], epz[i]])
        input_sequence = input_sequence.unsqueeze(0)

        target = torch.Tensor([Q2true[i]])
      
        # Forward pass
        output = model(input_sequence)
        loss = loss_fn(output, target)
      
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  # Test the model
def test(model, indexes, loss_fn):
    accuracy = 0
    total_loss = 0
    for i in indexes:
        input_sequence = torch.Tensor([eAngle[i], pth[i], eEnergy[i], epz[i]])
        input_sequence = input_sequence.unsqueeze(0)
        target = torch.Tensor([Q2true[i]])
        
        output = model(input_sequence)
        total_loss += loss_fn(output, target).item()

        print(target, output)
        accuracy += (target - output) / target
    return accuracy / len(indexes)

# Setup the model, data, loss function and optimizer
model = Network(4, 1)
data = [(list(range(10)), list(range(1, 11))), (list(range(10, 20)), list(range(11, 21)))]
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())


trainingData = int(len(xtrue) * 3/4)
testingData = len(xtrue) - trainingData

# Train the model
train(model,  list(range(trainingData)), loss_fn, optimizer, num_epochs=10)
print(test(model, list(range(trainingData, len(xtrue))), loss_fn))


0


c:\Users\eholt\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


1
2
3
4
5
6
7
8
9
tensor([10.0947]) tensor([[23.0369]], grad_fn=<AddmmBackward0>)
tensor([12.3840]) tensor([[22.3306]], grad_fn=<AddmmBackward0>)
tensor([13.8451]) tensor([[22.5180]], grad_fn=<AddmmBackward0>)
tensor([15.7927]) tensor([[21.7611]], grad_fn=<AddmmBackward0>)
tensor([18.1254]) tensor([[22.9853]], grad_fn=<AddmmBackward0>)
tensor([16.3314]) tensor([[22.4041]], grad_fn=<AddmmBackward0>)
tensor([25.2297]) tensor([[22.7665]], grad_fn=<AddmmBackward0>)
tensor([16.2508]) tensor([[22.3718]], grad_fn=<AddmmBackward0>)
tensor([12.7122]) tensor([[22.3637]], grad_fn=<AddmmBackward0>)
tensor([16.4243]) tensor([[22.8588]], grad_fn=<AddmmBackward0>)
tensor([13.3810]) tensor([[19.6909]], grad_fn=<AddmmBackward0>)
tensor([11.2181]) tensor([[22.3781]], grad_fn=<AddmmBackward0>)
tensor([20.4041]) tensor([[22.5905]], grad_fn=<AddmmBackward0>)
tensor([43.7662]) tensor([[22.9097]], grad_fn=<AddmmBackward0>)
tensor([26.7174]) tensor([[21.7766]], grad_fn=<AddmmBackward0>)
tensor([11.5690]) tens

In [49]:
import torch 
from torch import Tensor
import torch.nn as nn 
import torch.nn.functional as F 
from torch.optim import Adam

input_size = 4 # xe, xjb, xdiff, epz
learning_rate = 0.01 
output_size = 1          # Q^2



# Define neural network 
class Network(nn.Module): 
   def __init__(self, input_size, output_size): 
       super(Network, self).__init__() 
        
       self.layer1 = nn.Linear(input_size, 24) 
       self.layer2 = nn.Linear(24, 24) 
       self.layer3 = nn.Linear(24, output_size) 


   def forward(self, x): 
       x1 = F.relu(self.layer1(x)) 
       x2 = F.relu(self.layer2(x1)) 
       x3 = self.layer3(x2) 
       return x3 
 
# Instantiate the model 
model = Network(input_size, output_size) 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
print("The model will be running on", device, "device\n") 
model.to(device)    # Convert model parameters and buffers to CPU or Cuda 

# Function to save the model 
def saveModel(): 
    path = "./NetModel.pth" 
    torch.save(model.state_dict(), path)

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

def train(num_epochs): 
    best_accuracy = 0.0 
    accuracy = 0
    total = 0
     
    trainingData = int(len(xtrue) * 3/4)
    testingData = len(xtrue) - trainingData

    print("Begin training...") 
    for epoch in range(1, num_epochs+1): 
        total = 0 
 
        # Training Loop 
        for i in range(trainingData): 

        #for data in enumerate(train_loader, 0): 
            inputs = Tensor([eAngle[i], pth[i], eEnergy[i], epz[i]])
            outputs = Tensor([Q2true[i]])  # get the input and real species as outputs; data is a list of [inputs, outputs] 
            optimizer.zero_grad()   # zero the parameter gradients          
            predicted_outputs = model(inputs)   # predict output from the model 
            train_loss = loss_fn(predicted_outputs, outputs)   # calculate loss for the predicted output  
            train_loss.backward()   # backpropagate the loss 
            optimizer.step()        # adjust parameters based on the calculated gradients 
 

 
        # Validation Loop 
        with torch.no_grad(): 
            model.eval() 



            for i in range(trainingData, len(Q2true)): 
            #for data in enumerate(train_loader, 0): 
                inputs = Tensor([eAngle[i], pth[i], eEnergy[i], epz[i]])
                outputs = Tensor([Q2true[i]])  # get the input and real species as outputs; data is a list of [inputs, outputs] 
                predicted = model(inputs) 
               # The label with the highest value will be our prediction 
                #_, predicted = torch.max(predicted_outputs, 1) 

                print(predicted, outputs)
                accuracy += (predicted - outputs) / outputs

 
        # Calculate validation loss value 

                
        # Calculate accuracy as the number of correct predictions in the validation batch divided by the total number of predictions done.      
        accuracy /= testingData
        accuracy *= 100
 
        # Save the model if the accuracy is the best 
        if accuracy > best_accuracy: 
            saveModel() 
            best_accuracy = accuracy
         
        # Print the statistics of the epoch 
        print('accuracy:', accuracy)
train(4)

The model will be running on cpu device

Begin training...
tensor([3.8247e-25]) tensor([13.2543])
tensor([3.8247e-25]) tensor([15.9328])
tensor([3.8247e-25]) tensor([11.8639])
tensor([3.8247e-25]) tensor([18.5147])
tensor([3.8247e-25]) tensor([16.4069])
tensor([3.8247e-25]) tensor([14.2850])
tensor([3.8247e-25]) tensor([12.3360])
tensor([3.8247e-25]) tensor([10.7262])
tensor([3.8247e-25]) tensor([53.1886])
tensor([3.8247e-25]) tensor([10.1475])
tensor([3.8247e-25]) tensor([506.0345])
tensor([3.8247e-25]) tensor([20.4168])
tensor([3.8247e-25]) tensor([11.2638])
tensor([3.8247e-25]) tensor([32.7110])
tensor([3.8247e-25]) tensor([57.6493])
tensor([3.8247e-25]) tensor([10.0399])
tensor([3.8247e-25]) tensor([37.2381])
tensor([3.8247e-25]) tensor([195.9050])
tensor([3.8247e-25]) tensor([10.7974])
tensor([3.8247e-25]) tensor([530.1510])
tensor([3.8247e-25]) tensor([15.6677])
tensor([3.8247e-25]) tensor([15.1614])
tensor([3.8247e-25]) tensor([29.3966])
tensor([3.8247e-25]) tensor([14.7744])
te